# Introduction

## Présentation

Dans la mouvance NoSQL, Redis se classe dans la catégorie "data structure server"  (que l'on peut traduire en Français par "serveur de dictionnaire distant"). C'est  un système permettant de traiter, de manière très performante, des données sous la forme clés/valeurs. Contrairement à des solutions comme MemCache, Redis est capable de gérer la persistance des données en stockant, sur demande, son contenu sur disque.

La clé, qui peut être comparée au nom d'une variable dans un langage de programmation, permet d'identifier de manière unique la donnée. Lors de la création de votre application, vous allez rapidement vous rendre compte que le choix de cette clé n'est pas négligeable. C'est elle qui définira "la structure" de votre base de donnée.

Redis is an in memory data-structure store, and it supports several data structures and list is one of them.
The list data structure supported by Redis is implemented as a Linked List. This differs from the Python's implementation of the list.
In a linked list, operations of reading an element, insertion, deletion in the beginning or at the end takes constant time or in Big O notation O(1), as the head and tail addresses are always available prior to the operation.
To access an element positioned at Nth location in the list it, takes O(N) time.
Redis list is a list of strings.
From a Python program, a Redis list can be manipulated using Redis-Py, which is the Python interface for Redis.

## Les données 

Quant aux valeurs, elles peuvent appartenir aux types suivants:

- chaînes de caractères (strings)
- compteurs numériques (note: les données numériques seront stockées dans la base sous la forme de chaîne de - caractères) (atomic counters)
- listes (lists)
- tableaux (set) et tableaux ordonnés (sorted set)
- dictionnaires (hashes)

Comme on peut le voir, on retrouve la plupart des structures du langage Python. En étant un fervent admirateur, je me suis donc penché sur l'API Python Redis.

C'est parti pour un peu de bac à sable...

## Installation

Pour installer la librairie de Python permettant d'intéragir avec Redis

`pip install redis`

ou si on utilise pipenv comme dans notre cas 

`pipenv install redis`

# Let's play

In [5]:
import redis

In [15]:
redis_client = redis.StrictRedis(host='redis')

Pour tester la connexion on peut pinguer la base de données :

In [16]:
redis_client.ping()

True

Vous pouvez regarder toutes les configurations utilisé par Redis, beaucoup ne sont pas utiles à notre niveau mais vous pouvez voir par exemple, `database`, `maxclients`, `port` , `timeout` qui peuvent être utile de modifier pour différentes applications

In [9]:
redis_client.config_get()

{'dbfilename': 'dump.rdb',
 'requirepass': '',
 'masterauth': '',
 'cluster-announce-ip': '',
 'unixsocket': '',
 'logfile': '',
 'pidfile': '',
 'slave-announce-ip': '',
 'maxmemory': '0',
 'proto-max-bulk-len': '536870912',
 'client-query-buffer-limit': '1073741824',
 'maxmemory-samples': '5',
 'lfu-log-factor': '10',
 'lfu-decay-time': '1',
 'timeout': '0',
 'active-defrag-threshold-lower': '10',
 'active-defrag-threshold-upper': '100',
 'active-defrag-ignore-bytes': '104857600',
 'active-defrag-cycle-min': '25',
 'active-defrag-cycle-max': '75',
 'auto-aof-rewrite-percentage': '100',
 'auto-aof-rewrite-min-size': '67108864',
 'hash-max-ziplist-entries': '512',
 'hash-max-ziplist-value': '64',
 'list-max-ziplist-size': '-2',
 'list-compress-depth': '0',
 'set-max-intset-entries': '512',
 'zset-max-ziplist-entries': '128',
 'zset-max-ziplist-value': '64',
 'hll-sparse-max-bytes': '3000',
 'lua-time-limit': '5000',
 'slowlog-log-slower-than': '10000',
 'latency-monitor-threshold': '0'

## Stockage clé valeurs

Pour stocker une valeur correspondant à une clé 

In [10]:
KEY = "name"
VALUE = "yourname"
redis_client.set(KEY, VALUE)

True

True montre que le stockage s'est bien déroulé. 

Pour récupérer la valeur stocké il suffit d'appeler la méthode `GET` sur cette clé

In [17]:
redis_client.get(KEY)

b'yourname'

Vous pouvez voir que à l'aide du `b` que la réponse n'est pas exactement ce que l'on a stocké mais une représentation en bytes de notre chaine de charactères. 

In [18]:
assert redis_client.get(KEY) == VALUE, "Les valeurs ne correspondent pas"

AssertionError: Les valeurs ne correspondent pas

Pour cela, en python, il faut appeler la méthode `decode()` pour transformer des bytes en string 

In [19]:
assert redis_client.get(KEY).decode() == VALUE, "les valeurs ne correspondent pas"

On peut préciser au client de décoder les réponses et de les transformer directement en strings

In [22]:
redis_client = redis.StrictRedis(host="redis", port=6379, charset="utf-8", decode_responses=True)

In [23]:
assert redis_client.get(KEY) == VALUE

Les valeurs peuvent aussi être des entiers

In [24]:
KEY = "age"
VALUE = 20

redis_client.set(KEY, VALUE)

True

In [25]:
redis_client.get(KEY)

'20'

Vous pouvez alors utiliser la méthode `incr` pour incrémenter cet entier. Il faut faire attention puisque redis renvoie toujours les entiers comme des chaines de charactères.

In [26]:
redis_client.incr(KEY)

21

## Exemple 

In [27]:
from ipywidgets import widgets, interact
from IPython.display import display

In [28]:
KEY = "click_number"
DEFAULT_VALUE = 0

redis_client.set(KEY, DEFAULT_VALUE)

button = widgets.Button(description="Click me!")
def on_button_clicked(button):
    redis_client.incr(KEY)
    print(f"Le bouton à été clické {redis_client.get(KEY)} fois")
    button.description = "Click me more !"
    
button.on_click(on_button_clicked)
display(button)

Button(description='Click me!', style=ButtonStyle())

Le bouton à été clické 1 fois
Le bouton à été clické 2 fois
Le bouton à été clické 3 fois
Le bouton à été clické 4 fois
Le bouton à été clické 5 fois
Le bouton à été clické 6 fois
Le bouton à été clické 7 fois


## Les Listes

Redis permet aussi de stocker des valeurs de type `list`, pour cela nous avons à notre disposition un ensemble de méthodes :  

- La méthode `LPUSH`  permet d'ajouter un élément en tête de list la complexité est de O(1).  
- La méthode `RPUSH` permet d'ajouter un élément en queue de list la complexité est de O(1).  
- La méthode `LINSERT` permet d'ajouter un élément avant ou après la première occurence d'une certaine valeur, la complexité est de O(N) à part pour le premier et le dernier élèment.
- La méthode `LSET` permet de modifier la valeur correspondant à l'index spécifié, la complexité est de O(N) à part pour le premier et le dernier élèment.
- La méthode `RPOP` permet de supprimer un élément en queue de list la complexité est de O(1).
- La méthode `LPOP` permet de supprimer un élément en tête de list la complexité est de O(1).
- La méthode `LINDEX` permet d'afficher un élément en fonction de son index.
- La méthode `LREM` permet de supprimer un élément en fonction de son index.
- La méthode `LLEN` permet de récupérer la taille d'une liste.

Nous allons toutes les tester.

In [37]:
redis_client.lpush('language_list', "kotlin")
redis_client.lpush('language_list', "python")

4

On peut aussi ajouter plusieurs élément en même temps dans une liste

In [38]:
redis_client.lpush('language_list', "java", "c++")

6

In [41]:
f"La taille de la liste est {redis_client.llen('language_list')}"

'La taille de la liste est 6'

In [43]:
INDEX = 2
f"L'élément à l'index {INDEX} taille de la liste est '{redis_client.lindex('language_list', INDEX)}'"

"L'élément à l'index 2 taille de la liste est 'python'"

Le client renvoie la taille de la liste venant d'être mise à jour. On peut afficher tous les éléments de la liste en les retirant un par un.

In [33]:
while(redis_client.llen('language_list')!=0):

    print(redis_client.lpop('language_list'))

c++
java
python
kotlin


## Les dictionnaires 

Redis permet de stocker assez facilement des dictionnaires, on peut comme cela stocker des structures de données plus complexes. Pour cela nous avons plusieurs méthodes : 

- La méthode `HMSET` permet d'ajouter un dictionnaire à une clé
- La méthode `HSET` permet d'ajouter ou de modifier la valeur d'une clé d'un dictionnaire
- La méthode `HGET` permet d'ajouter ou de modifier la valeur d'une clé d'un dictionnaire
- La méthode `HGETALL` permet de récupérer toutes les clés valeurs d'un dictionnaire.
- La méthode `HKEYS` permet de récupérer toutes les clés d'un dictionnaire
- La méthode `HVALS` permet de récupérer toutes les valeurs d'un dictionnaire

In [45]:
doc = {
    "name":"yourname",
    "age":20,
    "city":"paris"
}

In [46]:
redis_client.hmset("user1", doc)

True

In [47]:
redis_client.hgetall("user1")

{'name': 'yourname', 'age': '20', 'city': 'paris'}

On peut modifier uniquement une clé du dictionnaire

In [120]:
redis_client.hset("user1", "city", "lyon")

0

In [121]:
redis_client.hgetall("user1")

{'age': '20', 'city': 'lyon', 'name': 'yourname'}

## Les pipelines

In [128]:
import timeit

Lorsque l'on appelle une méthode depuis le client python, chaqu'une d'entres elles fait un appel à Redis, ce qui peut prendre du temps. Les pipelines permettent de réduire ce temps en stockant toutes les opérations et en les applicant une seul fois sur la base. Cela permet d'augmenter grandement les performances en réduisant les appels et le passage par le réseau. 

In [143]:
N = 10000

In [144]:
def store_key_values(N):
    for i in range(N):
        redis_client.set(f"key_{i}", f"value_{i}")

In [145]:
%%timeit
store_key_values(N=N)

5.72 s ± 465 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [147]:
def store_key_values_with_pipelines(N):
    p = redis_client.pipeline()
    for i in range(N):
        p.set(f"key_{i}", f"value_{i}")

In [148]:
%%timeit
store_key_values_with_pipelines(N=N)

14.1 ms ± 393 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Exercice

Utiliser redis pour gérer les commandes d'un restaurant de burgers, pour simplifier les choses les clients pourront leurs ingrédients dans 3 types de produits :

- Pain: blanc, complet, aux céréales
- Garniture : salade, tomates , oignons, concombres
- Protéines : poulet, boeuf, agneau, tofu

Pour encore simplifier les commandes seront passées en amont.

Chaque ingrédient équivaut à un prix et un poids, avec chaque commande vous devrez calculer le poids et le prix que les clients devront régler. 

1) Stocker les informations concernant les ingrédients dans redis   
2) Créer la méthode permettant de créer aléatoirement un burger  
3) Créer la méthode permettant de calculer le poids et le prix du burger  
4) Créer une boucle pour réaliser et servir les burgers  

Pour vous aidez voici quelques données 

In [123]:
pains = {
    "blanc":{
        "poid":100,
        "prix": 2
    },
    "complet":{
        "poid":90,
        "prix": 3
    },
    "cereales":{
        "poid":95,
        "prix": 4
    }
}

In [124]:
garniture = {
    "salade":{
        "poid":50,
        "prix": 1
    }, "tomates":{
        "poid":60,
        "prix": 2
    } , "oignons":{
        "poid":10,
        "prix": 1.5
    }, "concombres":{
        "poid":30,
        "prix": 3
    }
}

In [125]:
proteine = {
    "poulet":{
        "poid":100,
        "prix": 6
    }, "boeuf":{
        "poid":150,
        "prix": 5
    } , "agneau":{
        "poid":120,
        "prix": 7
    }, "tofu":{
        "poid":80,
        "prix": 8
    }
}

In [1]:
def make_burger():
    pass

def process_price_weight():
    pass

def process_burger():
    pass

In [2]:
while True:
    burger, price, weight = process_burger()
    print(f'Le burger {burger} de {weight} gr à été préparé et servi il faut régler {price}€')

TypeError: 'NoneType' object is not iterable

In [ ]:
%load make_burger.py